In [35]:
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd
import numpy as np
import tblite.interface as tb
from xtb_optimization import generate_optimal_xyz
from gaussian_utils import generate_gaussian_input


In [29]:
smiles = "C1=CC=C2C(=C1)C=CO2"
trajectory, elements = generate_optimal_xyz(smiles, 'benzofuran.xyz')

------------------------------------------------------------
  cycle        total energy    energy error   density error
------------------------------------------------------------
      1     -24.19428441273  -2.4577167E+01   1.6353473E-01
      2     -24.22652702510  -3.2242612E-02   9.5132444E-02
      3     -24.21127942140   1.5247604E-02   5.3186971E-02
      4     -24.24069011004  -2.9410689E-02   1.4576981E-02
      5     -24.24181837305  -1.1282630E-03   5.4556760E-03
      6     -24.24203689296  -2.1851991E-04   1.5709325E-03
      7     -24.24204362941  -6.7364509E-06   8.6848252E-04
      8     -24.24204815738  -4.5279664E-06   3.7496469E-04
      9     -24.24204838244  -2.2505889E-07   1.1025620E-04
     10     -24.24204849982  -1.1737881E-07   3.6286447E-05
     11     -24.24204850383  -4.0159094E-09   2.0100172E-05
     12     -24.24204850583  -1.9982593E-09   5.8713224E-06
------------------------------------------------------------

 total:                             

In [31]:
import altair as alt
import polars as pl

df = pl.DataFrame(
    {
        "step": np.arange(len(trajectory)),
        "energy": np.asarray([energy for energy, _, _ in trajectory]),
        "gradient": np.asarray([np.abs(gradient).mean() for _, gradient, _ in trajectory]),
    }
)

base = alt.Chart(df).encode(
    x=alt.X("step", axis=alt.Axis(tickCount=len(df)), title="Step"),
)

gradient_line = base.mark_line(color="orange").encode(
    y=alt.Y("gradient", title="Mean Gradient (Hartree/Bohr)", scale=alt.Scale(zero=False)),
)
energy_line = base.mark_line(color="blue").encode(
    y=alt.Y("energy", title="Energy (Hartree)", scale=alt.Scale(zero=False)),
)
alt.layer(energy_line, gradient_line).resolve_scale(y="independent").properties(
    title="Benzofuran Optimization with xTB",
    width=600,
    height=400,
)

alt.LayerChart(...)

In [32]:
import json
import chemiscope


def write_chemiscope_input(
    name: str,
    elements: list[str],
    trajectory: list[tuple[np.ndarray, np.ndarray, np.ndarray]],
) -> dict:
    """Format geometry optimization data for Chemiscope."""
    return {
        "meta": {"name": name},
        "structures": [
            {
                "size": len(elements),
                "names": elements,
                "x": coordinates[:, 0].tolist(),
                "y": coordinates[:, 1].tolist(),
                "z": coordinates[:, 2].tolist(),
            }
            for _, _, coordinates in trajectory
        ],
        "properties": {
            "step": {
                "units": "",
                "target": "structure",
                "values": np.arange(len(trajectory)).tolist(),
            },
            "energy": {
                "units": "Hartree",
                "target": "structure",
                "values": [energy.item() for energy, _, _ in trajectory],
            },
            "gradient norm": {
                "units": "Hartree/Bohr",
                "target": "structure",
                "values": [np.abs(gradient).mean().item() for _, gradient, _ in trajectory],
            },
        },
    }


with open("geometry.json", "w") as fd:
    json.dump(
        write_chemiscope_input("Benzofuran Optimization with xTB", elements, trajectory),
        fd,
    )

chemiscope.show_input("geometry.json")

<ChemiscopeWidget(meta={'name': 'Benzofuran Optimization with xTB'}, structures=[{'size': 15, 'names': ['C', '…

In [36]:
generate_gaussian_input('benzofuran.xyz', 'benzofuran.com')

Gaussian input written to: benzofuran.com
